본인의 DATA SET load는 여기서 알아서 진행하도록 하자.
==

Encoder/Decoder train / test set 작성.
---
#### 현재 소스에서는 Decoder에서 Y값을 이동시키며 교사강요 시키는 소스가 아니므로, Y data set도 있어야함

#### 예시 
enc_train.shape : (batch_size, time_step, feature_len) <br />
dec_train.shape : (batch_size, time_step, wanted input decoder data len)  <br />
y_train.shape : (batch_size, time_step, predict data len)  <br />
 <br />
enc_test.shape : (batch_size, time_step, feature_len)  <br />
dec_test.shape : (batch_size, time_step, wanted input decoder data len)  <br />
y_test.shape : (batch_size, time_step, predict data len)  <br />

총 6개의 변수가 필요하다.

In [ ]:
%load_ext autoreload
%autoreload 2

from DARNN import DARNN

# time series
T = 24

# encoder hidden state cnt
m = 128

# decoder hidden state cnt
p = 128

batch_size = 2000

# 논문에서 encoder, decoder hidden state cnt는 서로 같아야 성능이 제일 좋다고 한다 (64, 128일때가 논문상 베스트)
model = DARNN(T=T, m=m, p=p, target_len=98)

In [ ]:
# epoch 돌아가는 쪽 소스를 수정할 거면, 굳이, tf dataset을 사용할 필요는 없다.

train_ds = (
    tf.data.Dataset.from_tensor_slices(
        (enc_train, dec_train, y_train)
    )
    .batch(batch_size)
#     .shuffle(buffer_size=20000)
    .prefetch(tf.data.experimental.AUTOTUNE)
)

test_ds = tf.data.Dataset.from_tensor_slices(
    (enc_test, dec_test, y_test)
).batch(batch_size)

In [ ]:
y_test.shape

In [ ]:
@tf.function
def train_step(model,inputs,labels,loss_fn,optimizer,train_loss):
    with tf.GradientTape() as tape:
        prediction = model(inputs,training=True)
        print(prediction.shape)
        print(labels.shape)
        loss = loss_fn(labels,prediction)
        print(loss)
    gradients = tape.gradient(loss,model.trainable_variables)
    optimizer.apply_gradients(zip(gradients,model.trainable_variables))
    train_loss(loss)

loss_fn = tf.keras.losses.MAE

optimizer = tf.keras.optimizers.Adam(0.001)
train_loss = tf.keras.metrics.Mean(name = 'train_loss')

for epoch in range(3000):
    for enc,dec,label in train_ds:
        inputs = [enc,dec]
        train_step(model,inputs,label,loss_fn,optimizer,train_loss)

    print(f"epoch : {epoch+1}, train_loss : {train_loss.result()}")
    train_loss.reset_states()

In [ ]:
test_loss = tf.keras.metrics.Mean(name = 'test_loss')

@tf.function
def test_step(model,inputs,labels,loss_fn,test_loss):
    prediction = model(inputs,training=True)
    loss = loss_fn(labels,prediction)
    test_loss(loss)
    return prediction

i=0
for enc,dec,label in test_ds:
    inputs = [enc,dec]
    pred = test_step(model,inputs,label,loss_fn,test_loss)
    if i==0:
        preds = pred.numpy()
        labels = label.numpy()
        i+=1
    else:
        preds = np.concatenate([preds, pred.numpy()],axis=0)
        labels = np.concatenate([labels,label.numpy()],axis=0)

print(test_loss.result())

In [ ]:
import matplotlib.pyplot as plt

import matplotlib.font_manager as fm
from matplotlib import rc

%matplotlib inline


In [ ]:
# Beta는 temporal attention에서 나온 시간에 대한 attention weight를 시각화할 수 있다.

enc,dec,label = next(iter(test_ds))
inputs = [enc, dec]
pred = model(inputs)

beta = []

# for문은 time step 만큼 돌린다.
for i in range(24):
    beta.append(np.mean(model.decoder.beta_t[:,i,0].numpy()))  # batch, T, 1

plt.bar(x = range(24), height=beta, color = 'orange')
plt.title("Beta")
plt.xlabel("time")
plt.ylabel("prob")
plt.show()

In [ ]:
# alpha는 Input Attention에서 나온 특징들에 대한 attention weight를 시각화 할 수 있다.

# variable_key는 특징값들이 들어가있으면된다.
variable_key = list(variable_dict.keys())
alpha = []
variables = []

# for문은 feature len만큼 돌린다.
for i in range(5):
    alpha.append(np.mean(model.encoder.alpha_t[:,0,i].numpy()))
    for key in variabel_key:
        # 같은 의미의 값이 여러개 있는경우 숫자를 붙혀줄 수 있는데, 특징이 유니크하게 1개인경우 해당 소스는 필요없다.
        if f"{i}" in variable_dict[key]:
            variables.append(f"{key}{i}")

plt.figure(figsize = (12,12))
plt.bar(x=variables,height=alpha, color = 'orange')
plt.title("alpha")
plt.xlabel("variables")
plt.xticks(rotation = 90)
plt.ylabel("prob")
plt.show()